### 📑 Dicionário de Dados - Visão Geral

A base utilizada neste projeto contém informações financeiras, comportamentais e históricas de crédito dos clientes. Abaixo, um resumo das principais colunas:

| Coluna                     | Descrição                                                        |
|---------------------------|-------------------------------------------------------------------|
| ID                        | Identificador único do registro                                   |
| Customer_ID               | Identificador do cliente                                          |
| Month                     | Mês de referência da transação                                    |
| Name                      | Nome do cliente                                                   |
| Age                       | Idade do cliente                                                  |
| SSN                       | Número de segurança social (formato com caracteres especiais)     |
| Occupation                | Profissão do cliente                                              |
| Annual_Income             | Renda anual                                                       |
| Monthly_Inhand_Salary     | Salário mensal disponível                                         |
| Num_Bank_Accounts         | Número de contas bancárias                                        |
| Num_Credit_Card           | Número de cartões de crédito                                      |
| Interest_Rate             | Taxa de juros                                                     |
| Num_of_Loan               | Quantidade de empréstimos ativos                                  |
| Type_of_Loan              | Tipos de empréstimos (texto com múltiplos valores)                |
| Delay_from_due_date       | Dias de atraso no pagamento                                       |
| Num_of_Delayed_Payment    | Número de pagamentos atrasados                                    |
| Changed_Credit_Limit      | Alteração no limite de crédito                                    |
| Num_Credit_Inquiries      | Número de consultas de crédito                                    |
| Credit_Mix                | Tipo de crédito utilizado (ruim, padrão, bom)                     |
| Outstanding_Debt          | Dívida pendente                                                   |
| Credit_Utilization_Ratio  | Percentual de utilização do limite de crédito                     |
| Credit_History_Age        | Tempo de histórico de crédito                                     |
| Payment_of_Min_Amount     | Se o pagamento mínimo foi realizado                               |
| Payment_Behaviour         | Comportamento de pagamento                                        |
| Monthly_Balance           | Saldo médio mensal                                                |
| Amount_invested_monthly   | Valor investido mensalmente                                       |
| Credit_Score              | Classificação do crédito (ruim, padrão, bom)                      |
 
Estas colunas passarão por etapas de limpeza, transformação e modelagem para alimentar o pipeline de classificação de clientes.


In [2]:
#Importações
from pyspark.sql.functions import col, sum, when, trim, regexp_replace, mean, min, max, count, when, percentile_approx, lit, expr, regexp_extract, round, translate, split, explode, first, create_map
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.types import DoubleType, IntegerType, StringType, NumericType
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from pyspark.sql.window import Window
from itertools import chain

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
#carregar base e visualizar dados
df_total = spark.read.csv("/FileStore/tables/client_credit_train.csv", header=True, inferSchema=True)
display(df_total)

In [ ]:
# Funções básicas

# Essa função retorna os valores, repetições e % dos dados de uma coluna específica
def get_valores(df, coluna):
    dados = df.groupBy(coluna).count()
    # Contagem dos valores e normalização (frequência percentual)
    total_count = df.count()
    dados_norm = dados.withColumn('percent', (F.col('count') / total_count) * 100)
    df_result = dados_norm.select(coluna, 'count', 'percent').orderBy(coluna)
    return df_result

# Essa função identifica, conta e calcula a frequencia % de valores nulos de um dataframe
def get_nulos(df):
    df_nulos = df.select([F.sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns])
    df_count = df.select([F.count(F.col(c)).alias(c) for c in df.columns])
    
    # Calculando o percentual de nulos diretamente
    result = []
    for column in df.columns:
        nulos = df_nulos.select(column).first()[0]
        total = df_count.select(column).first()[0]
        percent = (nulos / total) * 100 if total > 0 else 0
        percent_format = f"{percent:.2f}"
        result.append((column, total, nulos, percent_format))
    
    df_final = df.sparkSession.createDataFrame(result, ["Column", "Total", "Null_Count", "Null_Percentage"])
    return df_final

# Essa função identifica os valores não numerais de uma coluna
def get_nao_numericos(df, coluna):
    pattern = r'\D+'  # Qualquer caractere não numérico
    nao_numericos = df.filter(~F.col(coluna).rlike('^[0-9]+$')).select(coluna).distinct()
    return nao_numericos.rdd.flatMap(lambda x: x).collect()


# Essa funlção limpa a coluna substituindo caracteres não numéricos (exceto vírgula e ponto), troca vírgula por ponto, e converte para float.
def limpar_coluna_numerica(df, coluna):
    df = df.withColumn(coluna, regexp_replace(col(coluna), r'[^0-9,\.]', ''))
    df = df.withColumn(coluna, regexp_replace(col(coluna), ',', '.'))
    df = df.withColumn(coluna, col(coluna).cast('float'))
    return df

# Essa função retorna os valores únicos da coluna
def get_valores_unicos(df, coluna):
    df_unic = df.select(coluna).distinct()
    return df_unic

# Essa função retorna os valores unicos negativos
def get_valores_negativos(df, coluna):
    df_negativos = df.filter(col(coluna) < 0)
    return df_negativos

# Correção de valores negativos substituindo por 0
def corrigir_valores_negativos(df, coluna):
    df_corrigido = df.withColumn(
        coluna,
        when(col(coluna) < 0, 0).otherwise(col(coluna))
    )
    return df_corrigido

# Retonar caracteres especiais de uma coluna, numérica ou não
def get_caracteres_especiais(df, coluna):
    return df.filter(~col(coluna).rlike("^[a-zA-Z0-9 .,]*$"))

# Limpa a coluna com caracteres especiais
def limpar_caracteres_especiais(df, coluna):
    df = df.withColumn(coluna, regexp_replace(col(coluna), r'["""\'`]', ''))
    df = df.withColumn(coluna, regexp_replace(col(coluna), r'[^a-zA-Z0-9 .-]', ''))
    return df

# Padronizar valor colocando a moda entre os demais registros de um mesmo cliente
def padronizar(df, id_col, target_col):
    # Calcular a moda (idade mais frequente) por Customer_ID
    moda = (
        df.groupBy(id_col, target_col)
        .count()
        .withColumn("rank", F.row_number().over(Window.partitionBy(id_col).orderBy(F.desc("count"))))
        .filter(F.col("rank") == 1)
        .select(id_col, F.col(target_col).alias("moda"))
    )
    # Juntar com o dataframe original
    df_corrigido = (
        df.join(moda, on=id_col, how="left")
        .withColumn(
            target_col,
            F.when(F.col(target_col) != F.col("moda"), F.col("moda")).otherwise(F.col(target_col))
        )
        .drop("moda")
    )
    return df_corrigido

# Essa função arredonda valores double para 2 casas decimais após a vírgula
def arredondar_coluna(df, coluna):
    df_arredondado = df.withColumn(coluna, round(col(coluna), 2))
    return df_arredondado

# Função para dividir e criar novas colunas a partir de um registro separados por vírgula
def expandir_loans(df, coluna):
    # 1 - remover espaços e vírgulas duplicadas
    df_limpo = df.withColumn(
        coluna,
        regexp_replace(col(coluna), " and ", ", ")
    ) 
    # 2 - eparar os valores em uma lista, temporariamente
    df_separado = df_limpo.withColumn(
        "loan_list",
        split(col(coluna), ",")
    )
    # 3 explodir para pegar cada valor individualmente
    df_explodido = df_separado.withColumn("loan_type", explode("loan_list"))
    # 4 - remover espaços em branco ao redor
    df_explodido = df_explodido.withColumn("loan_type", trim(col("loan_type")))
    # 5 - pegar a lista de tipos únicos
    tipos_unicos = [row["loan_type"] for row in df_explodido.select("loan_type").distinct().filter(col("loan_type").isNotNull()).collect()]
    # 6 - para cada tipo, criar uma nova coluna
    df_final = df_separado
    for tipo in tipos_unicos:
        df_final = df_final.withColumn(
            tipo.replace(" ", "_"),  # Nome da coluna sem espaços
            F.when(F.array_contains(split(col(coluna), ","), tipo), lit(1)).otherwise(lit(0))
        )
    return df_final

    # Essa função calcula os limites IQR para identificar os outliers
def calcular_limites_iqr(df, colunas):
    limites = []
    for coluna in colunas:
        q1, q3 = df.approxQuantile(coluna, [0.25, 0.75], 0.01)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr
        limites.append((coluna, q1, q3, limite_inferior, limite_superior))
    return limites

# Essa função especifica remove alguns outlears 
def remover_outliers(df):
    df = df.filter(df["Interest_Rate"] <= 38)
    df = df.filter(df["Num_Credit_Card"] <= 11.5)
    df = df.filter(df["Num_Bank_Accounts"] <= 13)
    df = df.filter(df["Num_Credit_Inquiries"] <= 15.5)
    return df

# Essa fução substitui os valores da coluna Payment_Behaviour
def substituir_payment_behaviour_por_id(df):
    mapping = {
        "Low_spent_Small_value_payments": 0,
        "High_spent_Medium_value_payments": 1,
        "High_spent_Small_value_payments": 2,
        "Low_spent_Large_value_payments": 3,
        "Low_spent_Medium_value_payments": 4,
        "High_spent_Large_value_payments": 5
    }

    map_expr = create_map([lit(k) if i % 2 == 0 else lit(v)
                           for i, (k, v) in enumerate(mapping.items(), 0)])
    
    df = df.withColumn("Payment_Behaviour", map_expr[col("Payment_Behaviour")])
    return df




In [ ]:
# Diagnóstico Inicial do DataFrame

# Ver estrutura e tipos das colunas
print("\n Estrutura do DataFrame:")
df_total.printSchema()

# Ver número de linhas e colunas
linhas = df_total.count()
colunas = len(df_total.columns)
print(f"\n Dimensões do DataFrame: {linhas} linhas x {colunas} colunas")

# Verificar valores nulos por coluna
print("\n Valores pela função")
df_nulos = get_nulos(df_total)
df_nulos.show(27)

# Verificar valores em branco por coluna (strings vazias)
print(" Valores em Branco ('') por Coluna:")
df_total.select([
    sum(when(trim(col(c)) == "", 1).otherwise(0)).alias(c)
    for c in df_total.columns
]).show()

# Verificar registros duplicados
print("\n Total de registros duplicados:")
duplicados = df_total.groupBy(df_total.columns).count().filter("count > 1").count()
print(f"{duplicados} registros duplicados encontrados.")


In [ ]:
# Verificar valores únicos de cada coluna.

print("Valores unicos de cada coluna:")
for c in df_total.columns:
    print(f"Coluna: {c}")
    get_valores_unicos(df_total, c).show(truncate=False)

In [ ]:
# Conversão de colunas que deveriam ser numéricas e valores de String para double
colunas_para_converter = {
    "Age": IntegerType(),
    "Annual_Income": DoubleType(),
    "Outstanding_Debt": DoubleType(),
    "Changed_Credit_Limit": DoubleType(),
    "Num_of_Delayed_Payment": IntegerType(),
    "Amount_invested_monthly": DoubleType(),
    "Monthly_Balance": DoubleType(),
    "Num_of_Loan": IntegerType(),
    "Num_Credit_Inquiries": IntegerType()
}

# Limpeza de caracteres inválidos e conversão de tipo
for coluna, tipo in colunas_para_converter.items():
    df_total = limpar_coluna_numerica(df_total, coluna)
    df_total = df_total.withColumn(coluna, col(coluna).cast(tipo))

# Verificando novamente os tipos após conversão
df_total.printSchema()

# Dados após limpeza
df_total.select(list(colunas_para_converter.keys())).show(5)


In [ ]:
# Corrigir valores nulos nas colunas numéricas (exceto Credit_History_Age e Age pois serão tratadas posteriormente)
colunas_numericas = [
    "Monthly_Inhand_Salary",
    "Num_of_Delayed_Payment",
    "Changed_Credit_Limit",
    "Num_Credit_Inquiries",
    "Amount_invested_monthly",
    "Monthly_Balance"
]

# Lista para armazenar os resultados
resultados = []

# Loop para calcular estatísticas
for coluna in colunas_numericas:
    resumo = df_total.select(
        col(coluna).cast("double").alias(coluna)
    ).agg(
        count(coluna).alias("total_registros"),
        count(when(col(coluna).isNull(), True)).alias("nulos"),
        mean(coluna).alias("media"),
        percentile_approx(coluna, 0.5, 100).alias("mediana"),
        min(coluna).alias("minimo"),
        max(coluna).alias("maximo")
    ).withColumn("coluna", lit(coluna))

    resultados.append(resumo)

# União dos resultados
estatisticas_df = reduce(DataFrame.unionByName, resultados)

# Reordena as colunas para visualização
estatisticas_df = estatisticas_df.select("coluna", "total_registros", "nulos", "media", "mediana", "minimo", "maximo")

# Exibe a tabela de estatísticas
estatisticas_df.orderBy("coluna").show(truncate=False)


In [ ]:
# Calcular a mediana de cada coluna usando percentile_approx (equivalente ao 50º percentil) das colunas numéricas acima
medianas = {
    coluna: df_total.select(percentile_approx(coluna, 0.5).alias("mediana")).first()["mediana"]
    for coluna in colunas_numericas
}

# Substituir nulos pela mediana correspondente
for coluna in colunas_numericas:
    mediana = medianas[coluna]
    df_total = df_total.withColumn(
        coluna,
        when(col(coluna).isNull(), mediana).otherwise(col(coluna))
    )

In [ ]:
# Corrigir valores nulos nas colunas String
colunas_string = [f.name for f in df_total.schema.fields if isinstance(f.dataType, StringType)]

# Substituir valores nulos por "NA" nessas colunas
for coluna in colunas_string:
    df_total = df_total.withColumn(
        coluna,
        when(col(coluna).isNull(), "NA").otherwise(col(coluna))
    )


In [ ]:
# Verificação se há nulos por coluna
get_nulos(df_total).show(27)

In [ ]:
# A coluna "Name" foi verificado se há repetição ou valores "padronizados" como N/A ou *, pela frequencia de informações esta é uma variável 
# quantitativa.

# Verificando valores únicos e com caracteres especiais
df_name = get_valores(df_total, "Name")
df_name.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Name").show()
display(get_caracteres_especiais(df_total, "Name"))

# Janela particionada por cliente, ordenada (aqui usamos arbitrariamente por nenhuma coluna específica)
window_spec = Window.partitionBy("Customer_ID").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

# Padronizar nomes
df_total = padronizar(df_total, "Customer_ID", "Name")

#Existem diversos valores que destoam do normal, vamos manter o . e o -, os demais vão ser limpos
df_total = limpar_caracteres_especiais(df_total, "Name")

# Verificação após o tratamento
display(get_caracteres_especiais(df_total, "Name"))

In [ ]:
# A coluna Age tem outliers fácilmente identificáveis, aqui estamos filtrando os valores menores que 90 anos e substituindo
# pela mediana, nesse caso não há regra de negócio que trate isso, porém, pode ser identificado como fralde a movimentação desse
# titular em compras recorrentes, esta é uma coluna qualitativa.

# Verificando registros
df_age = get_valores(df_total, "Age")
df_age.orderBy(col("percent").desc()).show(truncate=False)

# Identificando os registros com idade maior que 90 anos
outliers_age = df_total.filter(col("Age") > 90)
print(f"Quantidade de outliers: {outliers_age.count()}")
display(df_total.filter(col("Customer_ID") == "CUS_0x39ce")) # Um cliente exemplo


# Vamos aplicar a padronização com a moda com os registros do mesmo cliente. Exemplo: o cliente possui um registro em agosto com a idade de 29 e outro registro em dezembro com a idade 455, vamos usar a moda para padronizar esses valores.
df_total = padronizar(df_total, "Customer_ID", "Age")
display(df_total.filter(col("Customer_ID") == "CUS_0x39ce"))

# Calcular a mediana de Age para os valores que não foram contemplados com a padronização.
mediana_age = df_total.select(
    percentile_approx("Age", 0.5, 100).alias("mediana")
).collect()[0]["mediana"]

print(f"A mediana: {mediana_age}")

# Substituir valores maiores que 90 pela mediana
df_total = df_total.withColumn(
    "Age",
    when(col("Age") > 90, lit(mediana_age)).otherwise(col("Age"))
)

# Verificação após tratamento
df_total.filter(col("Age") > 90).show()


In [ ]:
# Na coluna SSN existem valores com caracteres especiais, vamos substituilos por NA. SSN é uma variavel quantitativa, como um CPF, essa é uma coluna qualitativa.

# Verificando valores
get_valores_unicos(df_total, "SSN").show()
df_ssn = get_valores(df_total, "SSN")
df_ssn.orderBy(col("percent").desc()).show(truncate=False)

# Aplicar a padronização de acordo com o Customer_ID
df_ssn = df_total.filter(col("SSN") == "#F%$D@*&8")
print(f"Quantidade de outliers: {df_ssn.count()}")
display(df_ssn)
df_total = padronizar(df_total, "Customer_ID", "SSN")
display(df_total.filter(col("SSN") == "#F%$D@*&8"))

# Substituir os "#F%$D@*&8" que não foram contemplados pela padronização por "NA"
df_total = df_total.withColumn("SSN", when(col("SSN") == "#F%$D@*&8", "NA").otherwise(col("SSN")))

print("\n Verificação de valores #F%$D@*&8: ")
display(df_total.filter(df_total["SSN"]== "#F%$D@*&8").count())

In [ ]:
# Na coluna Occupation fornece um contexto demográfico e financeiro importante, visto que diferentes ocupações costumam ter diferentes níveis de renda, hábitos de consumo e perfis de risco, que são cruciais para a pontuação de crédito e outras análises financeiras. Existem valores "______" onde subsentende que representa o NA, essa coluna é definida como qualitativa.

# Verificando valores
df_occu = get_valores(df_total, "Occupation")
df_occu.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Occupation").show()

# Contar quantos registros têm na Occupation o valor "_______"
display(df_total.filter(df_total["Occupation"]== "_______").count())

# Substituindo "_______" por "NA"
df_total = df_total.withColumn(
    "Occupation",
    when(col("Occupation")== "_______", "NA").otherwise(col("Occupation"))
)

# Verificar valores após tratamento
display(df_total.filter(df_total["Occupation"]== "_______").count())

In [ ]:
# Na coluna Annual_Income representa a renda total que um cliente ganha em um ano. Esse valor é um indicador crucial da estabilidade financeira geral do cliente e de sua capacidade de pagar empréstimos ou administrar outras obrigações financeiras. Os valores nulos já foram corrigidos apenas precisamos arredondar o valor double para 2 casas decimais, essa coluna é definida como quantitativa

# Verificando valores
df_ani = get_valores(df_total, "Annual_Income")
df_ani.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Annual_Income").show(50)

# Verificando se há valores negativos
get_valores_negativos(df_total, "Annual_Income").show()

# Visualizar dados após tratamento
get_valores_unicos(df_total, "Annual_Income").show(50)

display(df_total)

In [ ]:
# Na coluna Monthly_Inhand_Salary representa o valor do salário que um cliente. Este é o salário líquido que o cliente leva para casa mensalmente e pode ser um indicador importante da renda disponível e da capacidade financeira do cliente. precisamos arredondar o valor double para 2 casas decimais, essa coluna também é uma coluna quantitativa

# Verificando valores
df_ani = get_valores(df_total, "Monthly_Inhand_Salary")
df_ani.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Monthly_Inhand_Salary").show(50)
get_caracteres_especiais(df_total, "Monthly_Inhand_Salary").show()

# Verificando se há valores negativos
get_valores_negativos(df_total, "Monthly_Inhand_Salary").show()

# Visualizar dados após tratamento
get_valores_unicos(df_total, "Monthly_Inhand_Salary").show(50)

display(df_total)

In [ ]:
# Na coluna Num_Bank_Accounts representa o número de contas bancárias que um cliente possui. Ao analisar os dados, foi identificado um valor incomum de -1. Como não é possível ter um número negativo de contas bancárias, esse valor provavelmente representa dados ausentes ou desconhecidos.

# Verificando valores
df_nbank = get_valores(df_total, "Num_Bank_Accounts")
df_nbank.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Num_Bank_Accounts").show(50)

# Verificando se há valores negativos
get_valores_negativos(df_total, "Num_Bank_Accounts").show()

# Corrigindo os valores negativos
df_total = corrigir_valores_negativos(df_total, "Num_Bank_Accounts")

# Visualizar dados após tratamento
get_valores_unicos(df_total, "Num_Bank_Accounts").show(50)
get_valores_negativos(df_total, "Num_Bank_Accounts").show()

In [ ]:
# Na coluna Num_Credit_Card representa a quantidade de cartões de crédito de um cliente, característica importante para categorização do cliete. Essa coluna é uma coluna qualitativa, semelhante à coluna anterior

# Verificando valores
df_nbank = get_valores(df_total, "Num_Credit_Card")
df_nbank.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Num_Credit_Card").show(50)

# Verificando se há valores negativos
get_valores_negativos(df_total, "Num_Credit_Card").show()

# Corrigindo os valores negativos
df_total = corrigir_valores_negativos(df_total, "Num_Credit_Card")

# Visualizar dados após tratamento
get_valores_unicos(df_total, "Num_Credit_Card").show(50)
get_valores_negativos(df_total, "Num_Credit_Card").show()

In [ ]:
# Na coluna Interest_Rate representa a taxa de juros aplicada aos empréstimos ou créditos tomados pelo cliente. Ela fornece informações cruciais sobre o custo do empréstimo para cada cliente. Essa coluna é uma coluna quantitativa

# Verificando valores
df_nbank = get_valores(df_total, "Interest_Rate")
df_nbank.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Interest_Rate").show(50)

# Verificando se há valores negativos
get_valores_negativos(df_total, "Interest_Rate").show()

# Corrigindo os valores negativos
df_total = corrigir_valores_negativos(df_total, "Interest_Rate")

# Visualizar dados após tratamento
get_valores_unicos(df_total, "Interest_Rate").show(50)
get_valores_negativos(df_total, "Interest_Rate").show()

In [ ]:

# Na coluna Num_of_Loan correspondente a quantidade de emprestimos, dado importante para categorização do cliente, existem valores com caracteres especiais e negativos, essa é uma coluna quantitativa

# Verificando valores
df_nbank = get_valores(df_total, "Num_of_Loan")
df_nbank.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Num_of_Loan").show()

# Verificar valores com caractere especial
display(get_caracteres_especiais(df_total, "Num_of_Loan"))

# Verificar valores negativos
display(get_valores_negativos(df_total, "Num_of_Loan"))

# Corrigir valores com caractere especial
df_total = limpar_caracteres_especiais(df_total, "Num_of_Loan")

# Corrigir valores negativos
df_total = corrigir_valores_negativos(df_total, "Num_of_Loan")

# Visualizar dados após tratamento
display(get_caracteres_especiais(df_total, "Num_of_Loan"))
display(get_valores_negativos(df_total, "Num_of_Loan"))

In [ ]:
# Backup do dataframe
df_total_bck_type_loan = df_total

In [ ]:
# Reset do dataframe
#df_total = df_total_bck_type_loan

In [ ]:
# Na coluna Type_of_Loan representa o(s) tipo(s) específico(s) de empréstimo que um cliente contratou. Essas informações ajudam a categorizar os diferentes produtos de empréstimo que um cliente pode ter e podem ser usadas para entender seu comportamento ou preferências financeiras. Precisei criar uma nova coluna para cada registro, pois os mesmos estavam agrupados em valores separados por "," são importantes pois definem o tipo do emprestimo contratado, coluna categorica.

# Verificando valores
df_type = get_valores(df_total, "Type_of_Loan")
df_type.orderBy(col("percent").desc()).show(truncate=False)
get_valores_unicos(df_total, "Type_of_Loan").show()

# Antes de expandir a coluna precisamos padroniza-las retirando os espaços em branco e o conector "and"
df_total = df_total.withColumn("Type_of_Loan", regexp_replace(col("Type_of_Loan"), "and", ""))
df_total = df_total.withColumn("Type_of_Loan", regexp_replace(col("Type_of_Loan"), " ", ""))
display(get_caracteres_especiais(df_total, "Type_of_Loan"))

# Criando as novas colunas e populando elas com inteiros
df_total = expandir_loans(df_total, "Type_of_Loan")

# Verificação após o tratamento
get_valores_unicos(df_total, "Type_of_Loan").show()
display(df_total)


In [ ]:
# Na coluna Num_of_Loan há uma diferença de acordo com o Type_of_Loan, exemplos há linhas que contém "0" na Num_of_loan mas existem 3 Type_of_loan registrados, com as novas colunas criadas conseguimos realizar um somatório do tipo e totalizar na coluna Num_of_Loan o valor correto.

display(df_total.filter(col("SSN") == "473-91-5845"))

# Lista das colunas que representam os tipos de empréstimos
tipo_emprestimos = [
    "PersonalLoan", "MortgageLoan", "AutoLoan", "NotSpecified", "PaydayLoan",
    "HomeEquityLoan", "StudentLoan", "DebtConsolidationLoan", "Credit-BuilderLoan"
]

# Soma das colunas para atualizar a coluna Num_of_Loan
df_total = df_total.withColumn(
    "Num_of_Loan",
    reduce(lambda a, b: a + b, [col(c) for c in tipo_emprestimos])
)

# Verificação após tratamento
display(df_total.filter(col("SSN") == "473-91-5845"))


In [ ]:
# Na coluna Delay_from_due_date representa o número de dias em que um pagamento está atrasado, o que é um indicador crítico do comportamento de pagamento e da responsabilidade financeira do cliente. É uma informação quantitativa, há valores negativos porém vou considerar como pagamentos adiantados

# Verificando valores
df_type = get_valores(df_total, "Delay_from_due_date")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Delay_from_due_date"))

In [ ]:
# Na coluna Num_of_Delayed_Payment representa o número de vezes que um cliente atrasou seu pagamento além da data de vencimento. Isso normalmente se refere à contagem total de pagamentos atrasados ​​que um cliente fez em empréstimos, cartões de crédito ou outras obrigações financeiras.
#Pode ser um recurso importante para modelos de pontuação de crédito, pois um número maior de pagamentos atrasados ​​geralmente indica um risco maior de inadimplência ou instabilidade financeira. que define o número de pagamentos atrasados, é uma coluna quantitativa, a mesma já passou pela limpeza de caracteres especiais.

# Verificando valores
df_type = get_valores(df_total, "Num_of_Delayed_Payment")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Num_of_Delayed_Payment"))

In [ ]:
# Na coluna Changed_Credit_Limit temos a mudança no limite de crédito, podemos considerar o aumento positivo como um acréscimo e negativo como débito. É uma coluna quantitativa.

# Verificando valores
df_type = get_valores(df_total, "Changed_Credit_Limit")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Changed_Credit_Limit"))

# Verificação após tratamento
display(df_total.select("Changed_Credit_Limit"))

In [ ]:
# Na coluna Num_Credit_Inquiries representa o número de consultas de crédito feitas no relatório de crédito de um cliente. Cada consulta normalmente ocorre quando um cliente solicita crédito, como um empréstimo ou cartão de crédito, e os credores solicitam a verificação do histórico de crédito do cliente. Um número maior de consultas de crédito pode indicar um risco maior de instabilidade financeira, pois consultas frequentes podem sugerir que o cliente está buscando múltiplas fontes de crédito. É uma coluna quantitativa

# Verificando valores
df_type = get_valores(df_total, "Num_Credit_Inquiries")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Num_Credit_Inquiries"))

In [ ]:
# Na coluna Credit_Mix representa a variedade de tipos de crédito que um cliente utiliza, como cartões de crédito e empréstimos. Uma combinação de crédito equilibrada pode afetar positivamente a pontuação de crédito de um cliente, demonstrando sua capacidade de administrar diferentes tipos de crédito. Existem 3 tipos de crédito: Good, Bad, Standard, os valores diferentes disso estão substituidos por "Uncategorized". É uma coluna qualitativa. 

# Verificando valores
df_type = get_valores(df_total, "Credit_Mix")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Credit_Mix"))

#Substituindo valores "_"
df_total = df_total.withColumn("Credit_Mix", regexp_replace(col("Credit_Mix"), "_", "Uncategorized"))

#Verificação de valores após o tratamento
display(df_total.select("Credit_Mix"))


In [ ]:
# Na coluna Outstanding_Debt representa o total de dívidas não pagas de um cliente, incluindo saldos de empréstimos, cartões de crédito e outras linhas de crédito. Ela fornece uma visão clara das obrigações financeiras atuais do cliente, o que é fundamental para avaliar o risco de crédito e determinar sua capacidade de gerenciar mais dívidas. É uma coluna quantitativa.

# Verificando valores
df_type = get_valores(df_total, "Outstanding_Debt")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Outstanding_Debt"))

# Verificação após o tratamento
display(df_total.select("Outstanding_Debt"))

In [ ]:
# Na coluna Credit_Utilization_Ratio representa a relação entre a dívida pendente atual de um cliente e seu crédito total disponível. Mede quanto do crédito disponível do cliente está sendo utilizado, expresso em porcentagem. É uma coluna quantitativa

# Verificando valores
df_type = get_valores(df_total, "Credit_Utilization_Ratio")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Credit_Utilization_Ratio"))

# Verificação após o tratamento
display(df_total.select("Credit_Utilization_Ratio"))


In [ ]:
# Na coluna Credit_History_Age representa a duração total do histórico de crédito de um cliente. Precisa ser transformada em uma coluna numérica. Está é uma coluna quantitativa

# Verificando valores
df_type = get_valores(df_total, "Credit_History_Age")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Credit_History_Age"))

#Extrair dos valores das strings
df_total = df_total.withColumn("credit_years", regexp_extract(col("Credit_History_Age"), r"(\d+)\s+Years", 1).cast("int"))
df_total = df_total.withColumn("credit_months", regexp_extract(col("Credit_History_Age"), r"(\d+)\s+Months", 1).cast("int"))

# Calcular o total de meses
df_total = df_total.withColumn("Credit_History_Months", (col("credit_years") * 12 + col("credit_months")))

# Remover as colunas temporárias
df_total = df_total.drop("credit_years", "credit_months")

# Visualizar valores convertidos
df_total.select("Credit_History_Age", "Credit_History_Months").show(10, truncate=False)

In [ ]:
# Substituir os valores Nulos da coluna Credit_History_Months
df_total = df_total.withColumn(
    "Credit_History_Months",
    when(col("Credit_History_Months").isNull(), 0).otherwise(col("Credit_History_Months"))
)

# Verificação após substituição
df_total.select("Credit_History_Months").distinct().show(truncate=False)

In [ ]:
# Na coluna Payment_of_Min_Amount representa se um cliente pagou o valor mínimo exigido em suas parcelas de crédito ou empréstimo. Existem valores NM ao qual não aparecem em nenhuma outra coluna, então vamos supor que seja "No"
get_valores_unicos(df_total, "Payment_of_Min_Amount").show()
display(df_total.filter(df_total["Payment_of_Min_Amount"] == "NM").count())

# Substituindo "NN" por "No"
df_total = df_total.withColumn(
    "Payment_of_Min_Amount",
    when(col("Payment_of_Min_Amount") == "NM", "No").otherwise(col("Payment_of_Min_Amount"))
)
print("\n Verificação de valores NM: ")
display(df_total.filter(df_total["Payment_of_Min_Amount"] == "NM").count())

In [ ]:
# Na coluna Total_EMI_per_month mostra quanto um cliente paga em parcelas mensais do empréstimo, incluindo principal e juros. Valores de EMI mais altos significam mais dívidas, o que pode afetar sua capacidade de assumir novos empréstimos ou gerenciar outras despesas. Esta é uma coluna quantitativa

# Verificando valores
df_type = get_valores(df_total, "Total_EMI_per_month")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Total_EMI_per_month"))

# Verificação após o tratamento
display(df_total.select("Total_EMI_per_month"))

In [ ]:
# Na coluna Amount_invested_monthly temos o valor investido mensal por cliente por mês. Refletindo uma "garantia" para o crédito, essa é uma coluna quantitativa

# Verificando valores
df_type = get_valores(df_total, "Amount_invested_monthly")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Amount_invested_monthly"))

# Verificação após o tratamento
display(df_total.select("Amount_invested_monthly"))


In [ ]:
# Na coluna Payment_Behaviour representam os padrões de gastos e pagamentos do cliente. Ela indica se o cliente gasta e faz pagamentos em valores pequenos, médios ou grandes. Esse recurso ajuda a entender como o cliente gerencia suas finanças em termos de gastos e pagamentos.m. Esta é uma coluna qualitativa

# Verificando valores
df_type = get_valores(df_total, "Payment_Behaviour")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Payment_Behaviour"))

# Substituir o valor '!@9#%8' por NA na coluna Payment_Behaviour
df_total = df_total.withColumn(
    "Payment_Behaviour",
    when(col("Payment_Behaviour") == "!@9#%8", "NA").otherwise(col("Payment_Behaviour"))
)

# Verificação após substituição
display(df_total.select("Payment_Behaviour").distinct())

In [ ]:
# Na coluna Monthly_Balance reflete o saldo restante na conta de um cliente ao final de cada mês, após a contabilização de todas as despesas e receitas. Ele fornece insights sobre a estabilidade e a gestão financeira do cliente. Essa é uma coluna quantitativa

# Verificando valores
df_type = get_valores(df_total, "Monthly_Balance")
df_type.orderBy(col("percent").desc()).show(truncate=False)
display(get_caracteres_especiais(df_total, "Monthly_Balance"))

# Verificação após o tratamento
display(df_total.select("Monthly_Balance"))

In [ ]:
# O recurso Credit_Score categoriza os clientes em Good, Poor, Standard com base em sua capacidade de crédito. No mundo real das finanças, isso ajuda os bancos a avaliar riscos, aprovar empréstimos e definir taxas de juros. Pontuações de crédito mais altas indicam menor risco, resultando em melhores condições de empréstimo, enquanto pontuações mais baixas sugerem maior risco.

# Para o conjunto de dados e o modelo de ANN, o Credit_Score é a variável-alvo que o modelo busca prever. Ao analisar características como renda, dívida e comportamento de pagamento, o modelo aprende a classificar os clientes nessas categorias. Isso permite uma melhor avaliação de riscos e tomada de decisões, auxiliando as instituições financeiras a tomarem decisões informadas relacionadas a crédito.

# Verificação de dados
get_valores_unicos(df_total, "Credit_Score").show()

Dada a natureza financeira dos dados, esses valores discrepantes podem representar variações legítimas nos perfis financeiros dos clientes.
Portanto, nenhuma ação imediata será tomada em relação a esses valores discrepantes. Se necessário, após avaliar o desempenho do modelo, ajustes poderão ser considerados para aprimorar ainda mais os resultados.

Além dos valores discrepantes, o conjunto de dados também contém atributos com alta assimetria,  Num_Bank_Accounts, Num_Credit_Inquiries, e Interest_Rate.

Essas distribuições assimétricas sugerem que os dados não estão distribuídos uniformemente, com muitos valores agrupados em um lado.
Embora a assimetria possa afetar o desempenho de certos modelos, ela pode não necessariamente prejudicar o processo de aprendizado de uma RNA, que é mais robusta a tais irregularidades em comparação com modelos lineares.

Vou criar um novo data frame voltado para análise ANN enquanto o df_total será reservado para a análise de dados. No novo data frame por enquanto, os atributos assimétricos não serão transformados, pois lidar com eles pode não ser crucial para uma RNA. Entretanto, se for constatado que a assimetria afeta o desempenho do modelo, transformações como a escala logarítmica podem ser consideradas. Também serão dropadas as colunas que não afetam o modelo como ID, Costumer_ID, Name, SSN. 

No df_total os outliers serão transformados se nencessário.

In [ ]:
# Criação do novo data frame voltado para ANN
df_total_ann = df_total

In [ ]:
# Iniciando verificação dos Outliers

# Seleciona apenas colunas numéricas
numeric_cols = [field.name for field in df_total.schema.fields if isinstance(field.dataType, NumericType)]

# Aplica o describe() somente às colunas numéricas
desc_df = df_total.select(numeric_cols).describe()

desc_transposed = (
    desc_df.toPandas()
    .set_index('summary')
    .transpose()
    .reset_index()
    .rename(columns={'index': 'feature'})
)

df_desc = spark.createDataFrame(desc_transposed)
display(df_desc)


In [ ]:
# Vamos passar pela análise de IQR as seguintes colunas com um desvio padrão muito alto com seus valores máximos respectivamente: as cilunas: Annual_Income = 2.4198062E7 Precisamos verificar o caso, se o valor alto também é presente em outras colunas, já as colunas Num_Bank_Accounts = 1798, Num_Credit_Card = 1499, Interest_Rate = 1499, Num_Credit_Inquiries = 2597 são valores claramente irreais.

colunas_outliers = {
    "Annual_Income",
    "Num_Bank_Accounts",
    "Num_Credit_Card",
    "Interest_Rate",
    "Num_Credit_Inquiries"
}

limites_iqr = calcular_limites_iqr(df_total, colunas_outliers)

for coluna, q1, q3, lim_inf, lim_sup in limites_iqr:
    print(f"{coluna}:\n  Q1 = {q1:.2f}, Q3 = {q3:.2f}, IQR = {q3 - q1:.2f}\n  Limite Inferior = {lim_inf:.2f}, Limite Superior = {lim_sup:.2f}\n")



In [ ]:
df_annual_backup = df_total

In [ ]:
# Vamos analisar a coluna Annual_Income primeiro, ela possui valores que betem e não batem com outras, como Monthly_Inhand_Salary, Monthly_Balance entre outras, para usa-la vou aplicar uma correção usando o Monthly_Inhand_Salary * 12 em uma nova coluna. Isso vai ser aplicado apenas para os registros que estão com o acima do limite superior no IQR.

# Verificação de valores
df_total.orderBy(col("Annual_Income").desc()).show(5)
df_annual = df_total.filter(col("Annual_Income") > 150340.99)
display(df_annual.orderBy(col("Annual_Income").desc()))

# Correção de valores
df_total = df_total.withColumn(
    "Annual_Income",
    when(col("Annual_Income") > 150340.99, col("Monthly_Inhand_Salary") * 12)
    .otherwise(col("Annual_Income"))
)

# Verificação após correção
display(df_total.orderBy(col("Annual_Income").desc()))

In [ ]:
df_bck = df_total

In [ ]:
display(df_total)
display(df_bck)

In [ ]:
# Adequiar coluna Payment_of_Min_Amount para processamento ANN, vamos separa os dataframes
get_valores_unicos(df_total, "Payment_of_Min_Amount").show()

# Substituir 'Yes' por 0 e 'No' por 1 na coluna Payment_of_Min_Amount
df_total_ann = df_total.withColumn(
    "Payment_of_Min_Amount",
    when(col("Payment_of_Min_Amount") == "Yes", 0)
    .when(col("Payment_of_Min_Amount") == "No", 1)
    .otherwise(col("Payment_of_Min_Amount"))
)

# Filtrando os dados não numéricos na coluna 'Payment_of_Min_Amount'
df_non_numeric = df_total.filter(col("Payment_of_Min_Amount").rlike("^[0-9]+$"))

# Mostrar os resultados
df_non_numeric.select(col("Payment_of_Min_Amount")).show(10)

df_total_ann.select(col("Payment_of_Min_Amount")).distinct().show()

A partir dessa parte vamos separar os dataframes, o "df_total" será voltado para a Análise Exploratória dos Dados. O "df_total_ann" será voltado para algoritmos de Machine Learn. 
Dito isso vamos descarregar o "df_total" em uma base de dados local para consulta.

In [ ]:
from pyspark.sql.functions import concat_ws

df_total.coalesce(1).write.mode("overwrite").option("header", True).csv("/dbfs/FileStore/df_total_csv_4")

In [ ]:
import pandas as pd
df_pd = df_total.toPandas()
df_pd.to_csv("/dbfs/FileStore/df_total_csv_4/df_total.csv", index=False)

/dbfs/FileStore/df_total_csv_4




Exploratory Data Analysis (EDA)


Colunas: ID, Customer_ID, Name, SSN

As colunas de identificação (ID, Customer_ID, Name, SSN) não são diretamente úteis em tarefas de classificação, embora forneça identificação e podem aumentar a complexidade do modelo em vez de melhorar seu desempenho.
Portanto, essas colunas serão descartadas ao final da etapa de limpeza dos dados.